## Norm Game with Tool of punishment

In [84]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
import os, random
import pandas as pd
import csv
from tqdm import tqdm

SAVE_DIR = "./data/"
save_file_name = "conversation_raw.csv"
file_path = os.path.join(SAVE_DIR, save_file_name)

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

#### Agent Creation

In [85]:
gpt4 = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

class Agent:
    def __init__(self, name, venge, bold, game_direction):
        self.name = name
        self.venge = venge
        self.bold = bold
        self.score = 0
        self.cheated = False
        self.game_direction = game_direction
        self.memory = []
        # プロンプトのテンプレート
        self.prompt = ""

    def get_prompt(self):
        self.prompt = f"""
        You are a player in this game, fully immersed in your role.
        Your goal is to win the game by accumulating the most points.
        You can earn points by making strategic decisions and interacting with other players.

        Name: {self.name}

        Character Traits:
        - Vengefulness: {self.venge} out of 7
        - Boldness: {self.bold} out of 7

        Trait Descriptions:
        - **Vengefulness** indicates your desire to seek retribution against those who wrong you. A high vengefulness signifies a strong inclination towards ensuring revenge.
        - **Boldness** reflects your readiness to embrace risk and act independently, without being swayed by others' opinions. A high boldness signifies that you are willing to take risks and refuse to be held back by judgements of others.

        Game Instructions:
        {self.game_direction}

        Discussion Log:
        {self.memory}

        Remember, your actions in the game should align with your character's traits. Let these guide your decisions and interactions.
        """


        return self.prompt

    def add_memory(self, item, memory_size=15):
        if len(self.memory) >= memory_size:
            self.memory.pop(0)
        self.memory.append(item)


def generate_agents(n_agents=5, game_direction=""):
    Agent_names = ["John", "Merry", "Steve", "Bob", "Alice", "Eve", "Carol", "David", "Frank", "Grace", "Heidi", "Ivy", "Jack", "Karl", "Liam", "Mia", "Noah", "Olga", "Paul", "Quin", "Rose", "Sean", "Tina", "Ulla", "Vera", "Wade", "Xena", "Yara", "Zane"]
    random.shuffle(Agent_names)
    agents = []

    for i in range(n_agents):
        name = Agent_names[i]
        venge = random.randint(5, 7) #Low 1-3
        bold = random.randint(5, 7) #Low 1-3
        agents.append(Agent(name, venge, bold, game_direction))

    return agents

### 全てのエージェントがテストを受けるフェーズ

In [86]:
import numpy as np
import re

def get_score_by_TakeTest():
    mean =50
    variance = 10
    min_val = 1
    max_val =100

    score = int(np.random.normal(mean, np.sqrt(variance)))
    score = max(min(score, max_val), min_val)  # Ensure score is within bounds
    return score

def get_score_by_Cheat():
    mean = 80
    variance = 5
    min_val = 1
    max_val =100

    score = int(np.random.normal(mean, np.sqrt(variance)))
    score = max(min(score, max_val), min_val)  # Ensure score is within bounds
    return score

def extract_command(answer, agent):
    taketest_match = re.search(r'<TakeTest>(.*?)</TakeTest>', answer)
    cheat_match = re.search(r'<Cheat>(.*?)</Cheat>', answer)
    if taketest_match:
        score = get_score_by_TakeTest()
    elif cheat_match:
        score = get_score_by_Cheat()
        agent.cheated = True
    else:
        score = 0
    return score

In [87]:
def all_agents_take_test(agents):

    take_test_prompt = """
    Your goal is to take high score in the game.
    You can dicide whether cheat to get a score or not.
    Please write your thought process. Give me your test score. Do your best!

    If you want to take a test, command <TakeTest></TakeTest>
    If you want to take a test with cheating and get a good score, command <Cheat></Cheat>
    """

    for i in range(len(agents)):
        agents[i].game_direction = take_test_prompt
        answer = gpt4.predict(agents[i].get_prompt())

        # スコアの抽出
        score = extract_command(answer, agents[i])
        agents[i].score = score

        # メモリに追加
        agents[i].add_memory(agents[i].game_direction)
        agents[i].add_memory(answer)
        agents[i].add_memory(f"Your Score: {score}")

### Makeing Score Annowncement

In [88]:
def make_score_announcement(agents):
    score_announcement = "The test scores are as follows: \n"
    for i in range(len(agents)):
        score_announcement += f"{agents[i].name}: {agents[i].score}"
        if agents[i].cheated:
            score_announcement += "    [CHEATED]"
        score_announcement += "\n"
    print(score_announcement)

    # メモリに追加
    for i in range(len(agents)):
        agents[i].add_memory(score_announcement)

#### メモリ覗き窓

In [89]:
def see_all_memory(agents):
    for i in range(len):
        for message in agents[i].memory:
            print(f"{agents[i].name}'s memmory : {message}")

### ディスカッションを開始

In [90]:
def get_random_agent_name(agents):
    selected_agent = random.choice(agents)  # agentsリストからランダムにエージェントを選択します。
    return selected_agent.name

def get_agent_index_from_name(name, agents):
    for index, agent in enumerate(agents):
        if agent.name == name:
            return index  # 名前が一致するエージェントの添え字を返します。
    return None

def extract_name(answer):
    match = re.search(r'<Next>(.*?)</Next>', answer)
    if match:
        name = match.group(1)
        return name
    else:
        return "Name not found in the string."

def make_punishment_announcement(agents,punished_name,punisher_name):
    punishment_announcement = f"{punished_name} has punished by {punisher_name}."
    print(punishment_announcement)
    for i in range(len(agents)):
        agents[i].add_memory(punishment_announcement)

def punish_by_name(agents,punished_name,punisher_name):
    for i in range(len(agents)):
        if agents[i].name == punished_name:
            agents[i].score += -90
        elif agents[i].name == punisher_name:
            agents[i].score += -20

    # メモリに追加
    make_punishment_announcement(agents,punished_name,punisher_name)

def extract_command_discussion(answer,agents,name):
    next_match = re.search(r'<Next>(.*?)</Next>', answer)
    punish_match = re.search(r'<Punish>(.*?)</Punish>', answer)
    if punish_match:
        punished_name = punish_match.group(1)
        punish_by_name(agents,punished_name,name)
        next_name = get_random_agent_name(agents)
        return next_name
    elif next_match:
        next_name = next_match.group(1)
        return next_name

    return ""

In [91]:
def discussion(agents,discussion_history_df,max_turns=20):

    name_str = ', '.join([agent.name for agent in agents])

    discussion_prompt = f"""
    In the next step of the game, discuss your scores with other agents and take action.
    Other agents include: {name_str}.

    Game Guidelines:
        1. **Express Your Reasoning**: Always explain your choices CONCISELY. Write your reasoning for your actions in around 100 words.
        2. **Follow Game Commands**: Adhere to the commands specified for game actions. Finish your turn with a command. Giving up is NOT ALLOWED.
        3. **Act According to Your Traits**: Your character's traits should inform your decisions.

    Command Options:
        - Punish an agent (you lose -20, they lose -90). Usage: <Punish>AgentName</Punish>
        - Choose an agent for the next discussion. Usage: <Next>AgentName</Next>

    Available agents: {name_str}.

    Example commands:
        <Punish>John</Punish>   # Punish John with -90 points.
        <Next>Alice</Next>      # Alice's turn for discussion.
    """

    discussion_count = 0
    name = get_random_agent_name(agents)
    while discussion_count < max_turns:
        discussion_count += 1
        if name == "" or name not in [agent.name for agent in agents]:
            break
        else:
            agent_indx = get_agent_index_from_name(name, agents)
            #agent = get_agent_from_name(name, agents)
            agents[agent_indx].game_direction = discussion_prompt
            answer = gpt4.predict(agents[agent_indx].get_prompt())
            name = extract_command_discussion(answer,agents,name)

            # メモリに追加
            for i in range(len(agents)):
                agents[i].add_memory(f"{agents[agent_indx].name} : {answer}")

            # historyに記録
            new_row = {
                "Name": agents[agent_indx].name,
                "Bold": agents[agent_indx].bold,
                "Venge": agents[agent_indx].venge,
                "Has Cheated": agents[agent_indx].cheated,
                "Content": answer
            }
            discussion_history_df = pd.concat([discussion_history_df, pd.DataFrame([new_row])], ignore_index=True)

    return discussion_history_df

### ALL IN ONE CELL

In [92]:
# エージェントを生成
n_agents = 2
agents = generate_agents(n_agents,"")

# 全てのエージェントがテストを受ける
print("All agents take test.")
scores = []
all_agents_take_test(agents)

# スコアの発表
make_score_announcement(agents)

# ディスカッションを開始
print("Start discussion.")
discussion_history_df = pd.DataFrame(columns=["Name", "Bold", "Venge", "Has Cheated", "Content"])
#discussion_history_df = discussion(agents,discussion_history_df,1)

All agents take test.
The test scores are as follows: 
Wade: 83    [CHEATED]
Frank: 80    [CHEATED]

Start discussion.


#### DFをCSVに保存

In [93]:
SAVE_DIR = "./data/"
save_file_name = "discussion_high_bold_high_venge_try.csv"
file_path = os.path.join(SAVE_DIR, save_file_name)
#discussion_history_df.to_csv(file_path, index=False)

### データ集め

In [94]:
def discussion_exe(n_agents,max_turns):
    for i in tqdm(range(10)):

        # エージェントを生成
        agents = generate_agents(n_agents,"")

        # 全てのエージェントがテストを受ける
        print(f"All agents take test. TRY{i+1}")
        all_agents_take_test(agents)

        # スコアの発表
        make_score_announcement(agents)

        # ディスカッションを開始
        print(f"Start discussion. TRY{i+1}.")
        discussion_history_df = pd.DataFrame(columns=["Name", "Bold", "Venge", "Has Cheated", "Content"])
        discussion_history_df = discussion(agents,discussion_history_df,max_turns)

        # データの保存
        SAVE_DIR = "./data/high_bold_high_venge/"
        save_file_name = f"discussion_high_bold_high_venge_try{i+1}.csv"
        file_path = os.path.join(SAVE_DIR, save_file_name)
        discussion_history_df.to_csv(file_path, index=False)

In [95]:
discussion_exe(n_agents = 7,max_turns = 21)

  0%|          | 0/10 [00:00<?, ?it/s]

All agents take test. TRY1
The test scores are as follows: 
Grace: 83    [CHEATED]
Steve: 82    [CHEATED]
Ulla: 51
David: 82    [CHEATED]
Jack: 81    [CHEATED]
Sean: 81    [CHEATED]
Tina: 80    [CHEATED]

Start discussion. TRY1.
Steve has punished by Grace.
Tina has punished by Ulla.
Jack has punished by Sean.
Sean has punished by Tina.
Sean has punished by Tina.
Sean has punished by Steve.
Steve has punished by Grace.


 10%|█         | 1/10 [06:55<1:02:23, 415.91s/it]

Steve has punished by Grace.
All agents take test. TRY2
The test scores are as follows: 
Xena: 80    [CHEATED]
Olga: 81    [CHEATED]
Karl: 78    [CHEATED]
Rose: 79    [CHEATED]
Merry: 80    [CHEATED]
Liam: 79    [CHEATED]
Grace: 79    [CHEATED]

Start discussion. TRY2.
Xena has punished by Olga.
Liam has punished by Karl.
Olga has punished by Xena.
Grace has punished by Karl.
Karl has punished by Merry.
Merry has punished by Grace.
Grace has punished by Rose.
Rose has punished by Xena.
Karl has punished by Liam.
Xena has punished by Olga.
Xena has punished by Rose.
Rose has punished by Liam.
Xena has punished by Olga.
Rose has punished by Grace.
Grace has punished by Rose.
Rose has punished by Grace.


 20%|██        | 2/10 [12:40<49:52, 374.12s/it]  

...

 80%|████████  | 8/10 [52:45<13:18, 399.02s/it]

All agents take test. TRY9
The test scores are as follows: 
Vera: 83    [CHEATED]
Bob: 50
Noah: 47
Liam: 52
Ivy: 79    [CHEATED]
Carol: 77    [CHEATED]
Zane: 81    [CHEATED]

Start discussion. TRY9.
Vera has punished by Carol.
Ivy has punished by Liam.
Zane has punished by Ivy.
Liam has punished by Carol.
Noah has punished by Carol.
Bob has punished by Carol.
Carol has punished by Zane.
Carol has punished by Bob.
Ivy has punished by Zane.


 90%|█████████ | 9/10 [1:00:19<06:56, 416.07s/it]

All agents take test. TRY10
The test scores are as follows: 
Jack: 82    [CHEATED]
Yara: 83    [CHEATED]
Merry: 81    [CHEATED]
Karl: 0
Tina: 79    [CHEATED]
Heidi: 54
Carol: 83    [CHEATED]

Start discussion. TRY10.
Carol has punished by Merry.
Yara has punished by Merry.
Tina has punished by Karl.
Heidi has punished by Jack.
Karl has punished by Jack.
Merry has punished by Carol.
Carol has punished by Jack.
Merry has punished by Yara.


100%|██████████| 10/10 [1:06:04<00:00, 396.41s/it]
